In [ ]:
!pip install biopython

In [ ]:
# #----------------------------------------蛋白质双序列比对-----------------------------------------

from Bio.Align import substitution_matrices
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [ ]:
# 计算一致度和相似度
def calculate_matches_similarities(aligned_seq1, aligned_seq2, matrix):
    matches = 0  # 完全匹配的残基数
    similarities = 0  # 相似残基数（根据矩阵）

    for a, b in zip(aligned_seq1, aligned_seq2):
        if a == '-' or b == '-':
            continue  # 忽略空位
        if a == b:
            matches += 1
        # 检查替换矩阵中的得分（BLOSUM62中正分表示相似）
        if (a, b) in matrix:
            score = matrix[(a, b)]
        else:
            score = matrix[(b, a)]  # 处理矩阵对称性
        if score > 0:
            similarities += 1
    return matches, similarities

In [ ]:
# 使用BLOSUM62得分矩阵
matrix = substitution_matrices.load("BLOSUM62")
print(matrix)

In [ ]:
protein1 = "ggrvln"
# 转为大写，如已经是大写则无需转换
protein1 = protein1.upper()
protein2 = "gkrvkv"
protein2 = protein2.upper()

In [ ]:
# 使用 BLOSUM62 矩阵，进行序列比对
alignments = pairwise2.align.globaldx(protein1, protein2, matrix)
# 打印比对结果
for a in alignments:
    print(format_alignment(*a))

In [ ]:
# 获取最佳比对结果（通常第一个）
best_alignment = alignments[0]
# 提取比对后的序列
aligned_seq1 = best_alignment.seqA
aligned_seq2 = best_alignment.seqB

# 计算一致度和相似度
matches, similarities = calculate_matches_similarities(aligned_seq1, aligned_seq2, matrix)
total = len(protein1)
identity = matches / total * 100
similarity = similarities / total * 100
print("一致度：", identity)
print("相似度：", similarity)

In [ ]:
#----------------------------------------核苷酸双序列比对-----------------------------------------

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [ ]:
# 定义两个核苷酸序列
seq1 = "ACACTTTCATTGTTTTACCGTTGCTCTGATTAATTGACGCTAAAGTCAGTAAAGTTAATCTCGTCAACACGGCACGCTACT"
seq2 = "CCGATGATCCTCATCGTAATCCAACCGAAACTTTACCTGATTCTGGCAGTCAAATCGGCTATCACAAAACAAGGATAAGGT"

# 使用全局比对算法进行比对，参数：序列1，序列2，匹配得分，错配得分，插入得分，插入扩展得分
# 自定义的得分矩阵
alignments = pairwise2.align.globalms(seq1, seq2, 2, -1, -0.5, -0.5)
seq1_aligned, seq2_aligned, score, begin, end = alignments[0]

In [ ]:
# 打印结果
print(format_alignment(*alignments[0]))
print("得分：", score)
print("序列1：", seq1)
print("序列2：", seq2)

In [ ]:
# 一致性：两个序列中相同位置的核苷酸相同的比例
# 该得分矩阵中一致度和相似度相同
identity = sum(1 for a, b in zip(seq1_aligned, seq2_aligned) if a == b) / len(seq1)
print("一致度：", identity)

In [ ]:
#-------------------------------利用在线BLAST从NCBI数据库进行序列比对---------------------------------------

from Bio.Blast import NCBIWWW
from Bio import SeqIO
import pandas as pd
from Bio.Blast import NCBIXML

In [ ]:
# 参数说明
# program: blastn（核酸 vs 核酸）、blastp（蛋白 vs 蛋白）、blastx（核酸转蛋白 vs 蛋白）
# database: nt（NCBI核酸库）、nr（NCBI蛋白库）、refseq_select（精选参考序列）
# expect (e-value): 期望值阈值，值越小表示匹配越显著（默认 0.001）
# hitlist_size: 返回的最大结果数量（默认 50）

def online_blast(query_file, blast_program='blastn', database='nt', evalue=0.001):
    # 读取查询序列
    query_seq = SeqIO.read(query_file, "fasta")

    # 提交在线BLAST请求
    print("正在提交BLAST请求...")
    result_handle = NCBIWWW.qblast(
        program=blast_program,  # 'blastn', 'blastp', 'blastx'等
        database=database,  # 'nt'（核酸库）或 'nr'（蛋白库）
        sequence=query_seq.seq,  # 查询序列
        expect=evalue,  # 期望值阈值
        hitlist_size=10,  # 返回结果数量
        format_type="XML"  # 输出格式（XML便于解析）
    )

    # 保存原始结果（可选）
    with open("blast_results.xml", "w") as out_file:
        out_file.write(result_handle.read())
    result_handle.close()

    return "blast_results.xml"



def blast_results_to_table(xml_file, output_csv="blast_results.csv"):
    data = []
    with open(xml_file) as f:
        blast_records = NCBIXML.parse(f)
        for record in blast_records:
            print(f"查询序列: {record.query}")
            for align in record.alignments:
                print(f"命中序列: {align.hit_id}")
                for hsp in align.hsps:
                    print(f"比对片段: \n{hsp.query}\n{hsp.match}\n{hsp.sbjct}")
                    row = {
                        "Query": record.query,
                        "Hit_ID": align.hit_id,
                        "E_value": hsp.expect,
                        "Score": hsp.score,
                        "Identity": hsp.identities / hsp.align_length * 100,
                        "Alignment_Length": hsp.align_length,
                        "Query_Start": hsp.query_start,
                        "Query_End": hsp.query_end,
                        "Hit_Start": hsp.sbjct_start,
                        "Hit_End": hsp.sbjct_end
                    }
                    data.append(row)

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    return df


In [ ]:
# 示例调用
result_file = online_blast("query.fasta")
df = blast_results_to_table("blast_results.xml")
print(df.head())